<a href="https://colab.research.google.com/github/glassresearch/PLT/blob/master/Python%20colab%20Georgia%20Tech/Window_Functions%20-%20Sum_Lead_Lag_FA25_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL Window Functions (AGGREGATIONS and LEAD/LAG)

In [1]:
!wget https://github.com/gt-cse-6040/bootcamp/raw/main/SQL/syllabus/university.db

--2025-10-14 15:22:45--  https://github.com/gt-cse-6040/bootcamp/raw/main/SQL/syllabus/university.db
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/SQL/syllabus/university.db [following]
--2025-10-14 15:22:45--  https://raw.githubusercontent.com/gt-cse-6040/bootcamp/main/SQL/syllabus/university.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 671744 (656K) [application/octet-stream]
Saving to: ‘university.db’

university.db       100%[===================>] 656.00K  --.-KB/s    in 0.006s  

2025-10-14 15:22:45 (109 MB/s) - ‘university.db’ saved [671744/671744]



In [2]:
import pandas as pd
from pprint import pprint, pformat

import sqlite3
conn=sqlite3.connect('university.db')


# Aggregations

The <b>SUM()</b> aggregation can be used both with and without the window option.

If you include the `OVER()` function, it allows you to create subtotals and running summaries based on the definition of your window. `SUM()` and `SUM() OVER()` is functionally the same query, so really we've been using a window function this whole time, but just for one extremely large window.

#### If we wanted to see a running total of scholarships earned by a student, or the running total of the semester hours earned by a student if the cumulative hours column didn't exist, you can calculate those with applying a sum with a window function parameter.

In [3]:
#scholarships by student
def RunningSum() -> str:
  return f"""
            SELECT
                sm.student_id,
                ss.scholarship_term,
                ss.scholarship_code,
                ss.scholarship_total,

                SUM(ss.scholarship_total) OVER
                                            (
                                                PARTITION BY sm.student_id
                                                ORDER BY ss.scholarship_term
                                            ) AS running_total
            FROM student_main sm
                INNER JOIN student_key sk
                    ON sm.student_id = sk.student_id
                INNER JOIN student_scholarship ss
                    ON sk.finance_id = ss.finance_id
                INNER JOIN scholarship_crosswalk sc
                    ON sc.scholarship_code = ss.scholarship_code
            --Included because the data includes financial transactions and we just want the scholarship totals
            WHERE scholarship_refund = 0
        """
display(pd.read_sql(RunningSum(),conn))


,student_id,scholarship_term,scholarship_code,scholarship_total,running_total
0,103972552,201901,PRE,1500,1500
1,103972552,201905,PRE,1500,3000
2,103972552,201908,PRE,1500,4500
3,103975474,201501,PRE,1500,1500
4,103975474,201505,PRE,1500,3000
...,...,...,...,...,...
1265,998275135,201601,FEG,2500,4500
1266,998275135,201601,USE,2000,4500
1267,998275135,201701,FEG,2500,7000
1268,998275135,201708,FEG,2500,9500


In [27]:
#cum semester hours earned
def RunningSum() -> str:
  # return f""" SELECT * FROM student_enrollment """
  return f"""
            SELECT
                sm.student_id,
                se.term,
                se.semester_hours_earned,

                SUM(se.semester_hours_earned) OVER (
                                                        PARTITION BY sm.student_id
                                                        ORDER BY se.term
                                                    ) AS running_total_hours

            FROM student_main sm
            INNER JOIN student_enrollment se
                ON sm.student_id = se.student_id
            ORDER BY sm.student_id, se.term
        """
display(pd.read_sql(RunningSum(),conn))

,student_id,term,semester_hours_earned,running_total_hours
0,100052749,201701,6,6
1,100052749,201705,8,14
2,100052749,201708,8,22
3,100636982,201501,8,8
4,100636982,201508,8,16
...,...,...,...,...
6690,998275135,202305,8,159
6691,998275135,202308,9,168
6692,998275135,202401,8,176
6693,998275135,202405,7,183


### While we only showed `SUM()` here, the above principles can be applied to all of the SQL Aggregate functions referenced previously, in the first Bootcamp SQL session.

We leave the application of these additional aggregate functions as an exercise for the students.

# LEAD / LAG

Our final example will be the `LEAD()` and `LAG()` functions.

They allow you to look `ahead (LEAD)` or `behind (LAG)` the current row to make comparisions between the two rows.

This is very helpful for `time series analysis` and `change detection`.

https://sqlite.org/windowfunctions.html


## Let's say that the administration is looking to find out which students are taking semester (or more) breaks.

#### Additionally, if the student has not been enrolled at all after that semester, the administration also wants to know that.

We can use the `julianday()` (also known as `datediff` in most other SQL versions) function and the `LEAD()` window function to see which students have a gap of more than one semester by comparing the current row's semester start date against the next row's semester start date.


### While what we are about to do may seem like a contrived example, it illustrates `TWO KEY FACTORS` that you will face in a Data Analytics role.

1. Data that needs to be analyzed **may not** be in a format that allows for direct analysis. As a result, `YOU, as the DA/DS` may have to make data assumptions and do data conversions in order to perform the required analysis.

    -- Any data assumptions you make MUST BE confirmed with the customers of your analysis, before performing them.

2. The tools that you are working with (in this case SQLITE) may not have direct functions to make the conversions you need. So you may need to put together sequences of operations, using the available functions.

### What we are about to do is one way to solve the requirement. There are many ways to do this, and this is simply to show one method.

### Finally, it is not unreasonable that a question such as this might be on an exam. The assumptions we are about to make would be provided, along with some form of hint to use the functions discussed below. This would be a 3-point question, for sure.

### So let's look at our data.

#### What we need to figure out is how to compute if a term has been skipped.

In [28]:
#LEAD Example
def terms() -> str:
  return f"""
            SELECT DISTINCT se.term
            FROM student_enrollment se
            ORDER BY se.term
            LIMIT 6
        """
display(pd.read_sql(terms(),conn))

,term
0,201501
1,201505
2,201508
3,201601
4,201605
5,201608


#### We can see that the terms are defined as a STRING format, as YYYYMM. As there is no DATE format in SQLite, we are going to have to do some work.

We also see that there are 3 terms in a year, starting in January, May, and August.

Because we don't have the actual start dates for each term, **we will make the assumption that each term starts on the first of that month.**

While this may not be realistic for New Year's Day, we will go with it, for ease of computations.

#### From this assumption, we can formulate a rule to compute if a student is registered in consecutive semesters (or not).

We can see that the semester start dates are all less than 6 months apart, so we can know that, if the date difference between consecutive semester start dates is greater than 180 days (approximately 6 months), then the student has skipped a sememster.

### What we want to do is use the `julianday()` function to determine if the student enrollments are greater than 180 days apart.

Top do this, we subtract the `julianday()` of the later semester from the `julianday()` of the prior semester, to return the number of days between the semester start dates. If this number is greater than 180, then we know that the student skipped a semester.

We can see from the below link that this function takes a date in the form of `yyyy-mm-dd` to return the julian day.

https://www.sqlitetutorial.net/sqlite-date-functions/sqlite-julianday-function/

### So we must convert the term, in the format `yyyymm` to the format `yyyy-mm-dd`, with the day being `01`.

We must then use that conversion in our date difference calculations.

### The SQL to do this conversion uses the SUBST and concatenation functions (|| -- double pipe), as follows:

`(SUBSTR(se.term,1,4) || '-' || SUBSTR(se.term,5,2) || '-01')`

This will return a STRING, such as `2015-01-01`, which will allow us to pass it to the `julianday` function.

### First, we will create a CTE in which we will compute three data points for the current and next terms, and make our calculations from them.

1. Term acronym, as defined in the `student_enrollment` table.
2. Term start date, in the form `yyyy-mm-dd`, using the SUBSTR and CONCAT functions.
3. Julianday for each term start date.

### Finally, our query will use the CTE to compute the date difference and return a string that tells if the student has skipped a semester, or if they have not enrolled at all, after that semester.

In [30]:
#LEAD Example
def LeadExampleLRH() -> str:
  return f"""
            WITH CTE_JULIAN_DAYS AS (
                            SELECT

                                sm.student_id,
                                --compute three pieces of information for current term
                                se.term
                                ,(SUBSTR(se.term,1,4) || '-' || SUBSTR(se.term,5,2) || '-01') as "term_start_date"
                                ,julianday((SUBSTR(se.term,1,4) || '-' || SUBSTR(se.term,5,2) || '-01'))  as "term_julianday"

                                --compute three pieces of information for next term
                                ,LEAD(se.term) OVER (
                                                    PARTITION BY sm.student_id
                                                    ORDER BY se.term
                                                ) AS next_term
                                ,LEAD((SUBSTR(se.term,1,4) || '-' || SUBSTR(se.term,5,2) || '-01')) OVER (
                                                    PARTITION BY sm.student_id
                                                    ORDER BY se.term
                                                ) AS next_term_start_date
                                ,LEAD(julianday((SUBSTR(se.term,1,4) || '-' || SUBSTR(se.term,5,2) || '-01'))) OVER (
                                                    PARTITION BY sm.student_id
                                                    ORDER BY se.term
                                                ) AS next_term_julianday
                            FROM student_main sm
                            INNER JOIN student_enrollment se
                                ON sm.student_id = se.student_id
                            --ORDER BY sm.student_id, se.term;
                )

                SELECT student_id
                        ,term
                        ,term_start_date
                        ,term_julianday
                        ,next_term
                        ,next_term_start_date
                        ,next_term_julianday
                        ,CASE
                            WHEN next_term_start_date IS NULL THEN 'Not enrolled after this term'
                            WHEN (next_term_julianday - term_julianday) > 180 THEN 'GAP > 6 months'
                            ELSE 'No significant gap'
                        END AS gap_flag
                FROM CTE_JULIAN_DAYS

        """
display(pd.read_sql(LeadExampleLRH(),conn))

,student_id,term,term_start_date,term_julianday,next_term,next_term_start_date,next_term_julianday,gap_flag
0,100052749,201701,2017-01-01,2457754.5,201705,2017-05-01,2457874.5,No significant gap
1,100052749,201705,2017-05-01,2457874.5,201708,2017-08-01,2457966.5,No significant gap
2,100052749,201708,2017-08-01,2457966.5,None,None,NaN,Not enrolled after this term
3,100636982,201501,2015-01-01,2457023.5,201508,2015-08-01,2457235.5,GAP > 6 months
4,100636982,201508,2015-08-01,2457235.5,201701,2017-01-01,2457754.5,GAP > 6 months
...,...,...,...,...,...,...,...,...
6690,998275135,202305,2023-05-01,2460065.5,202308,2023-08-01,2460157.5,No significant gap
6691,998275135,202308,2023-08-01,2460157.5,202401,2024-01-01,2460310.5,No significant gap
6692,998275135,202401,2024-01-01,2460310.5,202405,2024-05-01,2460431.5,No significant gap
6693,998275135,202405,2024-05-01,2460431.5,202408,2024-08-01,2460523.5,No significant gap


## Now let's say that the advisors want to reach out to students who have had their GPA drop in consecutive semesters.

### We can use the LAG() function, in a more straightforward manner, do compute this.

`LAG()` will give us the previous value (in this case, the term), and we can do our computations based on that.

In [34]:
#LAG Example
def LagExample() -> str:
  return f"""
            SELECT
                sm.student_id,
                LAG(se.term) OVER (
                                    PARTITION BY sm.student_id
                                    ORDER BY se.term
                                ) AS previous_term,
                se.term,
                LAG(se.semester_gpa) OVER (
                                            PARTITION BY sm.student_id
                                            ORDER BY se.term
                                        ) AS prev_semester_gpa,

                se.semester_gpa,
                se.semester_gpa - (LAG(se.semester_gpa) OVER (
                                            PARTITION BY sm.student_id
                                            ORDER BY se.term
                                            )
                                    ) AS gpa_raise_drop

            FROM student_main sm
            INNER JOIN student_enrollment se
                ON sm.student_id = se.student_id
            ORDER BY sm.student_id, se.term
        """
display(pd.read_sql(LagExample(),conn))

,student_id,previous_term,term,prev_semester_gpa,semester_gpa,gpa_raise_drop
0,100052749,None,201701,NaN,3.35,NaN
1,100052749,201701,201705,3.35,3.32,-0.03
2,100052749,201705,201708,3.32,2.74,-0.58
3,100636982,None,201501,NaN,3.03,NaN
4,100636982,201501,201508,3.03,2.47,-0.56
...,...,...,...,...,...,...
6690,998275135,202301,202305,3.35,2.61,-0.74
6691,998275135,202305,202308,2.61,1.50,-1.11
6692,998275135,202308,202401,1.50,2.50,1.00
6693,998275135,202401,202405,2.50,1.79,-0.71


## The below example will not be covered in the live session. It is included for additional knowledge on the use of these functions.

**_Extra Data Engineering Example_** <br>
While this class will not test you on data engineering coding, this is included as an additional example of a common use case of the <b>ROW_NUMBER()</b> window function:<br>

Row numbers can be super useful if you need an unique identifier in tables where you have duplicates. For instance, say your data engineering pipeline springs a leak and student scholarship information is duplicated or even triplicated. Rather than having to manually go through the table and delete the rows one by one, you can query the table using the row number function partioned by ID so the row number resets after each person and then delete where row_number > 1.

In [38]:
#code to populate table
def DEExample():
    return """
    --Pretend table creation
 WITH example(id, name) AS (
    VALUES
        (1, 'Alice'),
        (1, 'Alice'),
        (2, 'Bob'),
        (2, 'Bob'),
        (3, 'Charlie'),
        (3, 'Charlie'),
        (4, 'Diana'),
        (4, 'Diana'),
        (5, 'Eve'),
        (5, 'Eve')
)
SELECT
    id,
    name,
    ROW_NUMBER() OVER (PARTITION BY id ORDER BY id) AS row_num
FROM example;

    """
display(pd.read_sql(DEExample(), conn))

,id,name,row_num
0,1,Alice,1
1,1,Alice,2
2,2,Bob,1
3,2,Bob,2
4,3,Charlie,1
5,3,Charlie,2
6,4,Diana,1
7,4,Diana,2
8,5,Eve,1
9,5,Eve,2


If this was a real situation, you could then write a delete statement to delete from table where row_num = 2 and fix the data.

**ADDITIONAL READING**<br>
_This book is focused specifically on Microsoft SQL / T-SQL, but the book goes into great detail about the history and philosophy behind SQL itself and window functions. The syntax between T-SQL and SQLite is almost identical in this case._ <br>
T-SQL Window Functions: For data analysis and beyond, 2nd Edition <br>
https://learning.oreilly.com/library/view/t-sql-window-functions/9780135861554/
<br><br>
_His book_ T-SQL Fundamentals, 4th Edition _also goes into great detail about all things SQL._ <br>
https://learning.oreilly.com/library/view/t-sql-fundamentals-4th/9780138101930/ <br><br>

DataLemur is also an excellent resource for learning PostgreSQL, which is what SQLite is modeled on.<br>
https://datalemur.com/sql-tutorial/sql-aggregate-window-functions <br>
https://datalemur.com/sql-tutorial/query-order-of-execution
